# PyKX Query Components
Query all TorQ Components.


In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import boto3
import json
import datetime

import pykx as kx

from managed_kx import *
from env import *

from clusters import *

GATEWAY='gateway1'

In [2]:
# triggers credential get
session=None

if AWS_ACCESS_KEY_ID is None:
    print("Using Defaults ...")
    # create AWS session: using access variables
    session = boto3.Session()
else:
    print("Using variables ...")
    session = boto3.Session(
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        aws_session_token=AWS_SESSION_TOKEN
    )

# create finspace client
client = session.client(service_name='finspace', endpoint_url=ENDPOINT_URL)

Using Defaults ...


# List of Clusters

In [3]:
cdf = get_clusters(client, environmentId=ENV_ID)

all_clusters = [d['name'] for d in clusters if 'name' in d]

if cdf is not None:
    cdf = cdf[cdf['clusterName'].isin(all_clusters)]
#    cdf = cdf[cdf['status'].isin(['RUNNING'])]

display(cdf)

,clusterName,status,clusterType,capacityConfiguration,commandLineArguments,clusterDescription,lastModifiedTimestamp,createdTimestamp,databaseName,cacheConfigurations
0,discovery1,RUNNING,GP,None,"[{'key': 's', 'value': '2'}, {'key': 'jsonlogs', 'value': 'true'}, {'key': 'noredirect', 'value': 'true'}, {'key': 'procname', 'value': 'discovery1'}, {'key': 'proctype', 'value': 'discovery'}]",Created with create_all notebook,2024-03-18 20:22:32.476000+00:00,2024-03-18 19:34:28.565000+00:00,finspace-database,None
1,feed1,RUNNING,GP,None,"[{'key': 's', 'value': '2'}, {'key': 'jsonlogs', 'value': 'true'}, {'key': 'noredirect', 'value': 'true'}, {'key': 'procname', 'value': 'tradeFeed1'}, {'key': 'proctype', 'value': 'tradeFeed'}]",Created with create_all notebook,2024-03-18 20:36:18.125000+00:00,2024-03-18 19:51:26.204000+00:00,finspace-database,None
2,gateway1,RUNNING,GP,None,"[{'key': 's', 'value': '2'}, {'key': 'jsonlogs', 'value': 'true'}, {'key': 'noredirect', 'value': 'true'}, {'key': 'procname', 'value': 'gateway1'}, {'key': 'proctype', 'value': 'gateway'}]",Created with create_all notebook,2024-03-18 20:36:17.656000+00:00,2024-03-18 19:51:22.794000+00:00,finspace-database,None
3,hdb1,RUNNING,GP,None,"[{'key': 's', 'value': '4'}, {'key': 'jsonlogs', 'value': 'true'}, {'key': 'noredirect', 'value': 'true'}, {'key': 'procname', 'value': 'hdb1'}, {'key': 'proctype', 'value': 'hdb'}]",Created with create_all notebook,2024-03-18 20:22:31.186000+00:00,2024-03-18 19:34:34.442000+00:00,finspace-database,None
4,monitor1,RUNNING,GP,None,"[{'key': 's', 'value': '1'}, {'key': 'jsonlogs', 'value': 'true'}, {'key': 'noredirect', 'value': 'true'}, {'key': 'procname', 'value': 'monitor1'}, {'key': 'proctype', 'value': 'monitor'}]",Created with create_all notebook,2024-03-18 20:36:20.202000+00:00,2024-03-18 19:51:29.359000+00:00,finspace-database,None
5,rdb1,RUNNING,GP,None,"[{'key': 's', 'value': '2'}, {'key': 'jsonlogs', 'value': 'true'}, {'key': 'noredirect', 'value': 'true'}, {'key': 'procname', 'value': 'rdb1'}, {'key': 'proctype', 'value': 'rdb'}]",Created with create_all notebook,2024-03-18 20:22:32.112000+00:00,2024-03-18 19:34:31.625000+00:00,finspace-database,None


# Query the system through the Gateway

In [4]:
conn = get_pykx_connection(client, 
    environmentId=ENV_ID, clusterName=GATEWAY, 
    userName=KDB_USERNAME, boto_session=session)

queries = [
    '.gw.syncexec["meta trades";`rdb]',
    '.gw.syncexec["meta trades";`hdb]',
    '.gw.syncexec["select sum size by sym from trades";`rdb]',
    '.gw.syncexec["count trades";`rdb]',
    '.gw.syncexec["select [5] from trades";`rdb]',
    '.gw.syncexec["select [-5] from trades";`rdb]',
    '.gw.syncexec["select count i by date from trades";`hdb]', # FIXED
    '.gw.syncexec["select sum size, max price by 0D00:05 xbar time from trades where sym=`IBM";`hdb`rdb]',
    '.gw.syncexecj["select sum size by sym from trades";`rdb`hdb;sum]',
]

for q in queries:
    print(100*"=")
    print(f"Query: {q}")
    print(100*"-")
    display( conn(q).pd() )

Query: .gw.syncexec["meta trades";`rdb]
----------------------------------------------------------------------------------------------------


,t,f,a
c,,,
sym,b's',,g
time,b'p',,
src,b's',,
price,b'f',,
size,b'i',,


Query: .gw.syncexec["meta trades";`hdb]
----------------------------------------------------------------------------------------------------


,t,f,a
c,,,
date,b'd',,
sym,b's',,p
time,b'p',,
src,b's',,
price,b'f',,
size,b'i',,


Query: .gw.syncexec["select sum size by sym from trades";`rdb]
----------------------------------------------------------------------------------------------------


...

,x
date,
2014-04-21,29229
2014-04-22,26305
2014-04-23,32186


Query: .gw.syncexec["select sum size, max price by 0D00:05 xbar time from trades where sym=`IBM";`hdb`rdb]
----------------------------------------------------------------------------------------------------


,size,price
time,,
2014-04-21 08:00:00,106759,41.18
2014-04-21 08:05:00,105275,41.31
2014-04-21 08:10:00,53978,41.18
2014-04-21 08:15:00,76514,41.29
2014-04-21 08:20:00,114316,41.03
...,...,...
2014-04-23 16:25:00,138538,34.27
2024-03-18 20:20:00,6518,20.65
2024-03-18 20:25:00,120588,39.67


Query: .gw.syncexecj["select sum size by sym from trades";`rdb`hdb;sum]
----------------------------------------------------------------------------------------------------


,size
sym,
AAPL,25600204
CSCO,25715943
DELL,25644108
GOOG,26142213
IBM,25810790
MSFT,26259272
NOK,26097438
ORCL,25846713
YHOO,26130462


In [5]:
print( f"Last Run: {datetime.datetime.now()}" )

Last Run: 2024-03-18 20:38:02.842731
